In [65]:
#Fav / Dog Win percentage + odds filter
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
stake=100
datefilter='2023-01-01'

devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()

def hist_data_dog(surface, sex, odds_low,odds_high,date):
    data=pd.read_sql_query(f"Select distinct * from results_{surface}_1 where date > '{date}' ",con=devengine)#.to_clipboard(index=False)
    data["Fav_Odds"] = data.Fav_Odds.astype(float)
    data["Dog_Odds"] = data.Dog_Odds.astype(float)
    data['Fav_Win'] = data['Winner']==data['Fav']
    data['DogWin'] = data['Winner']!=data['Fav']
    data=data[(data['Fav_Odds']>=odds_low)&(data['Fav_Odds']<=odds_high)]
    data=data[data['Sex']==sex]
    return data

# Current date and time
current_datetime = datetime.datetime.now()#+ datetime.timedelta(days=-1)
# Convert to string
date_string = current_datetime.strftime("%Y-%m-%d")

In [66]:
Hard_womens=hist_data_dog('Hard','Womens',1,2,datefilter)
Hard_mens=hist_data_dog('Hard','Mens',1,2,datefilter)
temp=pd.concat([Hard_womens,Hard_mens]).sort_values(by='Date')
temp = temp.reset_index(drop=True)
temp['Fav_Loss'] = (temp['Fav_Win'] == 0).astype(int)
temp['Fav_Loss_Count'] = temp.groupby('Fav')['Fav_Loss'].cumsum()
temp['Dog_Win'] = (temp['Fav_Win'] == 0).astype(int)
temp['Dog_Win_Count'] = temp.groupby('Dog')['Dog_Win'].cumsum()
mind=temp[temp['Fav_Odds']<1.3].copy()
mind=mind[mind['Fav_Odds']>1]
mind=mind[mind['Date']>='2024-01-01']

favs=pd.read_csv('Fav_Hard.csv')
fav_list=favs['Name'].tolist()
mind=mind[(mind['Fav']).isin(fav_list)]
mind=mind[mind['dog_rank']>50]
len(mind[mind['Fav']==mind['Winner']])/len(mind),len(mind)
mind[['Fav','Dog','Fav_Odds','Winner','Fav_Win','dog_rank']].to_clipboard(index=False)



In [84]:
import pandas as pd

# Assuming 'temp' is your original dataframe
calc = temp.copy()

# Ensure 'Date' is a datetime type
calc['Date'] = pd.to_datetime(calc['Date'])

# Melt Fav and Dog results into long format for players and their wins
fav_df = calc[['Date', 'Fav', 'Fav_Win']].rename(columns={'Fav': 'Player', 'Fav_Win': 'Win'})
dog_df = calc[['Date', 'Dog', 'Dog_Win']].rename(columns={'Dog': 'Player', 'Dog_Win': 'Win'})

# Concatenate both datasets (i.e., wins for Fav and Dog)
player_results = pd.concat([fav_df, dog_df], ignore_index=True)

# Sort player results by Player and Date
player_results = player_results.sort_values(['Player', 'Date'])

# Create a dictionary to store the wins for each player
player_win_counts = {}

# Function to calculate rolling wins in the last 30 days for each player
def count_rolling_wins(player, match_date):
    if player not in player_win_counts:
        player_win_counts[player] = []
    
    # Filter player results within the last 30 days from the current match date
    recent_wins = player_results[(player_results['Player'] == player) & 
                                 (player_results['Date'] <= match_date) & 
                                 (player_results['Date'] > match_date - pd.Timedelta(days=30))]
    
    # Count the number of wins in this 30-day window
    return recent_wins['Win'].sum()

# Create new columns for the calculated rolling win counts for Fav and Dog
fav_wins_last_30 = []
dog_wins_last_30 = []

# Loop through each match in the original dataframe 'calc' to calculate the rolling wins for both Fav and Dog
for _, row in calc.iterrows():
    fav_wins = count_rolling_wins(row['Fav'], row['Date'])
    dog_wins = count_rolling_wins(row['Dog'], row['Date'])
    
    fav_wins_last_30.append(fav_wins)
    dog_wins_last_30.append(dog_wins)

# Add these calculated win counts as new columns in the 'calc' dataframe
calc['Fav_Wins_Last_30_Days_Before'] = fav_wins_last_30
calc['Dog_Wins_Last_30_Days_Before'] = dog_wins_last_30

# Ensure any missing counts are filled with 0 (in case the player has no matches in the last 30 days)
calc['Fav_Wins_Last_30_Days_Before'] = calc['Fav_Wins_Last_30_Days_Before'].fillna(0).astype(int)
calc['Dog_Wins_Last_30_Days_Before'] = calc['Dog_Wins_Last_30_Days_Before'].fillna(0).astype(int)


In [68]:
twst=calc[(calc['Fav'] == 'Rei Sakamoto') | (calc['Dog'] == 'Rei Sakamoto')]
twst=calc[(calc['Fav'] == 'Antonia Ruzic') | (calc['Dog'] == 'Antonia Ruzic')]
twst[twst['Date']>'2025-05-01'][['Date','Winner','Fav','Dog','Fav_Wins_Last_30_Days_Before','Dog_Wins_Last_30_Days_Before']]

,Date,Winner,Fav,Dog,Fav_Wins_Last_30_Days_Before,Dog_Wins_Last_30_Days_Before


In [85]:
calc['Predicted_Winner'] = calc['Fav_Wins_Last_30_Days_Before'] < calc['Dog_Wins_Last_30_Days_Before']
accuracy = (calc['Predicted_Winner'] == calc['Dog_Win']).mean()
print(f"Accuracy: {accuracy:.2%}")


Accuracy: 76.43%


In [87]:
stake=100
date_filter=calc[calc['Date']>'2023-01-01']
oddsfilter=date_filter[date_filter['Fav_Odds']>1].copy()
oddsfilter=oddsfilter[oddsfilter['Dog_Wins_Last_30_Days_Before']>oddsfilter['Fav_Wins_Last_30_Days_Before']+1]
oddsfilter=oddsfilter[oddsfilter['Sex']=='Womens']
oddsfilter['Profit'] = oddsfilter.apply(
    lambda row: (row['Dog_Odds'] - 1) * stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = oddsfilter['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(oddsfilter),len(oddsfilter[oddsfilter['Dog']==oddsfilter['Winner']])/len(oddsfilter))
oddsfilter[['Date','Dog_Odds','Winner','Dog','Fav','Dog_Win_Count','Fav_Loss_Count','Dog_Wins_Last_30_Days_Before','Fav_Wins_Last_30_Days_Before']]
#oddsfilter#.to_csv('_HardResults_Womens.csv')

Total Profit: 33335.0 347 0.7579250720461095


,Date,Dog_Odds,Winner,Dog,Fav,Dog_Win_Count,Fav_Loss_Count,Dog_Wins_Last_30_Days_Before,Fav_Wins_Last_30_Days_Before
104,2024-02-13,2.27,Karolina Pliskova,Karolina Pliskova,Anna Kalinskaya,1,1,3,0
147,2024-02-14,2.34,Karolina Pliskova,Karolina Pliskova,Linda Noskova,2,1,5,2
148,2024-02-14,2.69,Leylah Fernandez,Leylah Fernandez,Qinwen Zheng,2,1,3,1
151,2024-02-14,2.97,Lesia Tsurenko,Lesia Tsurenko,Ons Jabeur,1,1,2,0
153,2024-02-14,5.46,Katerina Siniakova,Katerina Siniakova,Cori Gauff,2,1,2,0
...,...,...,...,...,...,...,...,...,...
7843,2025-03-23,3.46,Naomi Osaka,Hailey Baptiste,Naomi Osaka,9,5,5,3
7891,2025-03-25,6.10,Magda Linette,Magda Linette,Coco Gauff,11,5,6,4
7926,2025-03-27,2.21,Maddison Inglis,Maddison Inglis,Erika Andreeva,5,9,4,0
7942,2025-03-28,1.95,Rebeka Masarova,Rebeka Masarova,Bernarda Pera,9,5,6,3


In [ ]:
stake=100
date_filter=calc[calc['Date']>'2025-01-01']
oddsfilter=date_filter[date_filter['Fav_Odds']>1].copy()
oddsfilter=oddsfilter[oddsfilter['Dog_Wins_Last_30_Days_Before']>oddsfilter['Fav_Wins_Last_30_Days_Before']+1]
oddsfilter=oddsfilter[oddsfilter['Sex']=='Mens']
oddsfilter['Profit'] = oddsfilter.apply(
    lambda row: (row['Dog_Odds'] - 1) * stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = oddsfilter['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(oddsfilter),len(oddsfilter[oddsfilter['Dog']==oddsfilter['Winner']])/len(oddsfilter))
oddsfilter[['Date','Dog_Odds','Winner','Dog','Fav','Dog_Wins_Last_30_Days_Before','Fav_Wins_Last_30_Days_Before']]
#oddsfilter#.to_csv('_HardResults_Mens.csv')

Total Profit: 11270.0 166 0.6626506024096386


,Date,Dog_Odds,Winner,Dog,Fav,Dog_Wins_Last_30_Days_Before,Fav_Wins_Last_30_Days_Before
5235,2025-01-03,2.49,Ethan Quinn,Ethan Quinn,Martin Landaluce,4,1
5413,2025-01-07,3.44,Brandon Nakashima,Yannick Hanfmann,Brandon Nakashima,4,1
5526,2025-01-08,1.97,Kimmer Coppejans,Kimmer Coppejans,Valentin Vacherot,4,1
5584,2025-01-08,2.12,Jakub Mensik,Jakub Mensik,Ben Shelton,3,0
5620,2025-01-09,2.37,Nishesh Basavareddy,Nishesh Basavareddy,Alex Michelsen,8,4
...,...,...,...,...,...,...,...
8169,2025-05-09,2.44,Alex Bolt,Alex Bolt,Constant Lestienne,4,1
8202,2025-05-22,2.64,Federico Cina,Eliakim Coulibaly,Federico Cina,6,2
8258,2025-06-01,2.01,Michael Zheng,Michael Zheng,Liam Draxl,4,0
8281,2025-06-05,2.20,Rio Noguchi,Rio Noguchi,Stefan Dostanic,4,1


In [72]:
stake=100
date_filter=calc[calc['Date']>'2025-01-01']
oddsfilter=date_filter[date_filter['Fav_Odds']>1.7].copy()
oddsfilter=oddsfilter[oddsfilter['Dog_Wins_Last_30_Days_Before']+1<oddsfilter['Fav_Wins_Last_30_Days_Before']]
oddsfilter=oddsfilter[oddsfilter['Sex']=='Mens']
oddsfilter['Profit'] = oddsfilter.apply(
    lambda row: (row['Fav_Odds'] - 1) * stake if row['Winner'] == row['Fav'] else -stake, axis=1
)

# Calculate total profit
total_profit = oddsfilter['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(oddsfilter),len(oddsfilter[oddsfilter['Fav']==oddsfilter['Winner']])/len(oddsfilter))
oddsfilter[['Date','Dog_Odds','Winner','Dog','Fav','Dog_Win_Count','Fav_Loss_Count','Dog_Wins_Last_30_Days_Before','Fav_Wins_Last_30_Days_Before']]
#oddsfilter.to_csv('_HardResults_Mens.csv')

Total Profit: 5030.0 105 0.8285714285714286


,Date,Dog_Odds,Winner,Dog,Fav,Dog_Win_Count,Fav_Loss_Count,Dog_Wins_Last_30_Days_Before,Fav_Wins_Last_30_Days_Before
5206,2025-01-02,1.96,Constant Lestienne,Valentin Vacherot,Constant Lestienne,2,5,0,2
5436,2025-01-07,2.06,Alibek Kachmazov,Alibek Kachmazov,Shintaro Mochizuki,9,8,1,4
5451,2025-01-07,1.87,Raul Brancaccio,Tristan Lamasine,Raul Brancaccio,2,0,0,2
5533,2025-01-08,1.96,Dominik Koepfer,Dominik Koepfer,Ethan Quinn,4,5,2,5
5552,2025-01-08,2.00,Facundo Diaz Acosta,Sebastian Baez,Facundo Diaz Acosta,1,2,0,3
...,...,...,...,...,...,...,...,...,...
8231,2025-05-29,1.94,Patrick Kypson,Mitchell Krueger,Patrick Kypson,7,4,0,2
8274,2025-06-04,1.96,Rafael Jodar,Rafael Jodar,Andres Andrade,1,3,1,3
8279,2025-06-04,1.97,Andres Martin,Aziz Dougaz,Andres Martin,7,1,0,3
8283,2025-06-05,1.87,Andres Martin,Rafael Jodar,Andres Martin,1,1,1,4


In [73]:
from sqlalchemy import text

#with devengine.connect() as conn:
#    conn.execute(text('DELETE FROM results_Hard_1 WHERE Date > "2025-06-07"'))
#    conn.commit()
#pd.read_sql_query('Select distinct* from AllMatches where date = "2025-06-08" and Sex like "Womens"',con=devengine)

In [74]:
tempo = temp.copy()
date_filter = pd.read_pickle(r'.\Hard_Today').drop(columns=['Fav_Loss_Count','Dog_Win_Count'])
date_filter['Date'] = date_string
calc=pd.concat([tempo, date_filter], ignore_index=True)


In [75]:
import pandas as pd


# Ensure 'Date' is a datetime type
calc['Date'] = pd.to_datetime(calc['Date'])

# Melt Fav and Dog results into long format for players and their wins
fav_df = calc[['Date', 'Fav', 'Fav_Win']].rename(columns={'Fav': 'Player', 'Fav_Win': 'Win'})
dog_df = calc[['Date', 'Dog', 'Dog_Win']].rename(columns={'Dog': 'Player', 'Dog_Win': 'Win'})

# Concatenate both datasets (i.e., wins for Fav and Dog)
player_results = pd.concat([fav_df, dog_df], ignore_index=True)

# Sort player results by Player and Date
player_results = player_results.sort_values(['Player', 'Date'])

# Create a dictionary to store the wins for each player
player_win_counts = {}

# Function to calculate rolling wins in the last 30 days for each player
def count_rolling_wins(player, match_date):
    if player not in player_win_counts:
        player_win_counts[player] = []
    
    # Filter player results within the last 30 days from the current match date
    recent_wins = player_results[(player_results['Player'] == player) & 
                                 (player_results['Date'] <= match_date) & 
                                 (player_results['Date'] > match_date - pd.Timedelta(days=30))]
    
    # Count the number of wins in this 30-day window
    return recent_wins['Win'].sum()

# Create new columns for the calculated rolling win counts for Fav and Dog
fav_wins_last_30 = []
dog_wins_last_30 = []

# Loop through each match in the original dataframe 'calc' to calculate the rolling wins for both Fav and Dog
for _, row in calc.iterrows():
    fav_wins = count_rolling_wins(row['Fav'], row['Date'])
    dog_wins = count_rolling_wins(row['Dog'], row['Date'])
    
    fav_wins_last_30.append(fav_wins)
    dog_wins_last_30.append(dog_wins)

# Add these calculated win counts as new columns in the 'calc' dataframe
calc['Fav_Wins_Last_30_Days_Before'] = fav_wins_last_30
calc['Dog_Wins_Last_30_Days_Before'] = dog_wins_last_30

# Ensure any missing counts are filled with 0 (in case the player has no matches in the last 30 days)
calc['Fav_Wins_Last_30_Days_Before'] = calc['Fav_Wins_Last_30_Days_Before'].fillna(0).astype(int)
calc['Dog_Wins_Last_30_Days_Before'] = calc['Dog_Wins_Last_30_Days_Before'].fillna(0).astype(int)


In [ ]:
today=calc[calc['Date']==date_string]
today=today[today['Fav_Odds']>1]
today=today[today['Dog_Wins_Last_30_Days_Before']>today['Fav_Wins_Last_30_Days_Before']+2]
womens=today[today['Sex']=='Womens']
womens[['Dog','Dog_Odds','Fav','Dog_Wins_Last_30_Days_Before','Fav_Wins_Last_30_Days_Before']]


,Dog,Dog_Odds,Fav,Dog_Wins_Last_30_Days_Before,Fav_Wins_Last_30_Days_Before


In [ ]:
today=calc[calc['Date']==date_string]
today=today[today['Fav_Odds']>1]
today=today[today['Dog_Wins_Last_30_Days_Before']>today['Fav_Wins_Last_30_Days_Before']+2]
mens=today[today['Sex']!='Womens']
mens[['Time','Dog','Dog_Odds','Fav','Fav_Odds','Dog_Wins_Last_30_Days_Before','Fav_Wins_Last_30_Days_Before']]

,Time,Dog,Dog_Odds,Fav,Fav_Odds,Dog_Wins_Last_30_Days_Before,Fav_Wins_Last_30_Days_Before


In [ ]:
today=calc[calc['Date']==date_string]
today=today[today['Fav_Odds']>1.7]
today=today[today['Dog_Wins_Last_30_Days_Before']+2<today['Fav_Wins_Last_30_Days_Before']]
mens_favs=today[today['Sex']!='Womens']
mens_favs[['Time','Dog','Dog_Odds','Fav','Fav_Odds','Dog_Wins_Last_30_Days_Before','Fav_Wins_Last_30_Days_Before']]

,Time,Dog,Dog_Odds,Fav,Fav_Odds,Dog_Wins_Last_30_Days_Before,Fav_Wins_Last_30_Days_Before
8366,04:10,Alexander Blockx,1.9,Liam Draxl,1.83,4,7


In [ ]:
today=calc[calc['Date']==date_string]
today=today[today['Fav_Odds']>1.7]
today=today[today['Dog_Wins_Last_30_Days_Before']+2<today['Fav_Wins_Last_30_Days_Before']]
womens_favs=today[today['Sex']=='Womens']
womens_favs[['Time','Dog','Dog_Odds','Fav','Fav_Odds','Dog_Wins_Last_30_Days_Before','Fav_Wins_Last_30_Days_Before']]

,Time,Dog,Dog_Odds,Fav,Fav_Odds,Dog_Wins_Last_30_Days_Before,Fav_Wins_Last_30_Days_Before


In [80]:
womens_favs[['Date','Time','Dog','Dog_Odds','Fav','Fav_Odds','Dog_Wins_Last_30_Days_Before','Fav_Wins_Last_30_Days_Before']].to_sql('hard_womens_favs',con=devengine,if_exists='append')
mens_favs[['Date','Time','Dog','Dog_Odds','Fav','Fav_Odds','Dog_Wins_Last_30_Days_Before','Fav_Wins_Last_30_Days_Before']].to_sql('hard_mens_favs',con=devengine,if_exists='append')
mens[['Date','Time','Dog','Dog_Odds','Fav','Fav_Odds','Dog_Wins_Last_30_Days_Before','Fav_Wins_Last_30_Days_Before']].to_sql('hard_mens',con=devengine,if_exists='append')
womens[['Date','Time','Dog','Dog_Odds','Fav','Fav_Odds','Dog_Wins_Last_30_Days_Before','Fav_Wins_Last_30_Days_Before']].to_sql('hard_womens',con=devengine,if_exists='append')

0